This notebook gives a simple example of how Google Earth Engine can be leveraged to extract additional information for the EUSEDcollab station points. To run this notebook will require the python packages (listed in the cell below) to be installed in a local environment in addition to a Google Earth Engine account. Introductory information can be found at the following site: https://geemap.org/. This example is based on the tutorial:
https://geemap.org/notebooks/43_extract_values_to_points/

In [2]:
import geemap
import os
import ee
import geopandas as gpd

Here we can define the path to the shapefile of EUSEDcollab monitoring station points. If this notebook is placed in the EUSEDcollab parent directory, the relevant file will be directly read. Otherwise the shapefile path will need to be specified.

In [ ]:
#define the local path to the EUSEDcollab directory 
dir_ = os.getcwd()
#define the sub-directory with the additional information
points_dir = os.path.join(dir_, 'ADDITIONAL_FILES')
in_shp = os.path.join(points_dir, 'EUSEDcollab_station_points.shp')
print(in_shp)

Initiate a map so that we can later visualise data.

In [ ]:
Map = geemap.Map()
Map

Add a SRTM layer to the map from Google Earth Engine

In [5]:
# Add Earth Engine dataset
dem = ee.Image('USGS/SRTMGL1_003')

# Set visualization parameters.
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

Map.addLayer(dem, vis_params, 'SRTM DEM', True, 1)

Here we create layers of the annual sum of the monthly precipitation and the mean annual average temperature. We can later extract the point values from these.

In [6]:
rainfall_collection = ee.ImageCollection('WORLDCLIM/V1/MONTHLY')
#get the sum of the average monthly precipitation
rainfall_year = rainfall_collection.select('prec').sum()
#get the mean temperature and unscale the value
temp_mean = rainfall_collection.select('tavg').mean().multiply(0.1)

Here we read in the EUSEDcollab points shapefile which defines the points at which we will extract the raster pixel values.

In [ ]:
in_fc = geemap.shp_to_ee(in_shp)
Map.addLayer(in_fc, {}, 'EUSEDcollab points')

The next 3 cells extract the raster pixel values and save them to a local .csv file. These can then be compiled and used alongside the database to provide auxillary information for additional analyses. Google Earth Engine contains a large number of ready-to-use datasets that can facilitate large scale analyses without downloading large amounts of data locally. A large number of tutorials in python can be found on the following webpage: https://geemap.org/

In [ ]:
out_csv = os.path.join(points_dir, 'dem_vals.csv')
geemap.extract_values_to_points(in_fc, dem, out_csv)

In [ ]:
precip_csv = os.path.join(points_dir, 'precip_vals.csv')
geemap.extract_values_to_points(in_fc, rainfall_year, precip_csv, scale = 1000)

In [ ]:
tmean_csv = os.path.join(points_dir, 'tmean_vals.csv')
geemap.extract_values_to_points(in_fc, temp_mean, tmean_csv, scale = 1000)